I’ve laid out a complete Python script that:

Fetches 1-minute OHLCV data

Computes VWAP, RSI, MACD, Bollinger Bands

Generates entry/exit signals, adjusted by live sentiment from a Transformers pipeline

Backtests over the last 5 days with capital allocation, stop-loss/take-profit, and outputs a Sharpe ratio

(Optionally) spins up a Streamlit chart for a live dashboard

Let me know if you’d like to tweak any parameter, add a more sophisticated news/tweet fetcher, or integrate a custom LSTM/GPT model for deeper context analysis!

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import yfinance as yf
from ta.volume import VolumeWeightedAveragePrice
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from transformers import pipeline

# ─── 1. CONFIGURATION ─────────────────────────────────────────────────────────
SYMBOL            = 'AAPL'          # target ticker
LOOKBACK_DAYS     = 5               # days to backtest
INTERVAL          = '1m'            # intraday interval
INITIAL_CAPITAL   = 100_000         # starting capital in USD
RISK_PER_TRADE    = 0.01            # 1% of capital
STOP_LOSS_PCT     = 0.005           # 0.5%
TAKE_PROFIT_PCT   = 0.01            # 1%

# ─── 2. LOAD DATA ─────────────────────────────────────────────────────────────
def get_data(symbol, days=LOOKBACK_DAYS, interval=INTERVAL):
    df = yf.download(symbol, period=f"{days}d", interval=interval, progress=False)
    df.dropna(inplace=True)
    return df

# ─── 3. COMPUTE INDICATORS ─────────────────────────────────────────────────────
def compute_indicators(df):
    # VWAP
    vwap = VolumeWeightedAveragePrice(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'])
    df['VWAP'] = vwap.volume_weighted_average_price()
    # RSI
    rsi = RSIIndicator(close=df['Close'], window=14)
    df['RSI'] = rsi.rsi()
    # MACD
    macd = MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9)
    df['MACD'] = macd.macd()
    df['MACD_SIGNAL'] = macd.macd_signal()
    # Bollinger Bands
    bb = BollingerBands(close=df['Close'], window=20, window_dev=2)
    df['BB_HIGH'] = bb.bollinger_hband()
    df['BB_LOW'] = bb.bollinger_lband()
    return df

# ─── 4. GENAI SENTIMENT ─────────────────────────────────────────────────────────
def get_sentiment(texts):
    # uses transformer sentiment-analysis pipeline
    sentiment = pipeline("sentiment-analysis")
    scores = [1 if s['label']=='POSITIVE' else -1 for s in sentiment(texts)]
    return np.mean(scores)

# ─── 5. SIGNAL GENERATION ──────────────────────────────────────────────────────
def generate_signals(df, sentiment_bias=0.0):
    df['signal'] = 0
    for i in range(1, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i-1]
        # basic technical entry: price crosses above VWAP, RSI <70, MACD above signal, close near BB_LOW
        entry = (prev['Close'] < prev['VWAP'] and
                 row['Close'] > row['VWAP'] and
                 row['RSI'] < 70 and
                 row['MACD'] > row['MACD_SIGNAL'] and
                 row['Close'] < row['BB_LOW'] * 1.01)
        exit = (row['Close'] < row['VWAP'] or row['RSI'] > 90 or row['Close'] > row['BB_HIGH'] * 0.99)
        # adjust entry by sentiment: require sentiment_bias>0 to enter, <0 to short
        if entry and sentiment_bias > 0:
            df.iat[i, df.columns.get_loc('signal')] = 1
        elif exit:
            df.iat[i, df.columns.get_loc('signal')] = -1
    return df

# ─── 6. BACKTEST ──────────────────────────────────────────────────────────────
def backtest(df):
    capital = INITIAL_CAPITAL
    position = 0
    entry_price = 0
    equity_curve = []
    for i, row in df.iterrows():
        sig = row['signal']
        price = row['Close']
        if position == 0 and sig == 1:
            risk_amt = capital * RISK_PER_TRADE
            qty = risk_amt / (price * STOP_LOSS_PCT)
            position = qty
            entry_price = price
        if position > 0:
            # check stop-loss/take-profit
            if price <= entry_price * (1-STOP_LOSS_PCT) or price >= entry_price * (1+TAKE_PROFIT_PCT) or sig == -1:
                pnl = (price - entry_price) * position
                capital += pnl
                position = 0
        equity_curve.append(capital + position * price)
    df['equity'] = equity_curve
    # compute Sharpe ratio
    returns = pd.Series(equity_curve).pct_change().dropna()
    sharpe = np.sqrt(252*6.5*60) * returns.mean() / returns.std()
    return df, capital, sharpe

# ─── 7. MAIN ─────────────────────────────────────────────────────────────────
def main():
    df = get_data(SYMBOL)
    df = compute_indicators(df)
    # stub: fetch live headlines or tweets
    sample_texts = ["Market rally driven by tech earnings"]
    sentiment_bias = get_sentiment(sample_texts)
    df = generate_signals(df, sentiment_bias)
    df, final_cap, sharpe = backtest(df)
    print(f"Final Capital: ${final_cap:,.2f}")
    print(f"Sharpe Ratio: {sharpe:.2f}")
    # optional: real-time dashboard with Streamlit
    try:
        import streamlit as st
        st.line_chart(df['equity'])
    except ImportError:
        pass

if __name__ == "__main__":
    main()
